In [ ]:
import hkvfewspy
import hkvsobekpy
import pandas
import altair
import hkvportal

pi = hkvfewspy.Pi() 
pi.setClient(wsdl='https://webviewer.hkvfewsscheldestromen.nl:8443/FewsPiService/fewspiservice?wsdl')

# get available locations
locations = pi.getLocations()
locations

# slice by column and row
locations.KST457.x

# get available parameters
parameters = pi.getParameters()
parameters

# slice by column
parameters.WNSHDB1

# initiate query object
query = pi.setQueryParameters()

locations.KST521.locationId

# define query parameters
query.parameterIds([parameters.WNSHDB1.id])
query.locationIds(['KST457', 'KST291', 'KST521'])
query.startTime(pandas.datetime(2019,5,20))
query.endTime(pandas.datetime(2019,6,3))

# request timeseries from FEWS given the query parameters
timeseries = pi.getTimeSeries(query)
timeseries.head()

# mask nodata values using the flag value
timeseries.value.mask(timeseries.flag == 8, inplace=True)
timeseries.head()

# setup an interactive line chart for the year 1953-1955
line_chart = altair.Chart(timeseries.reset_index()).mark_line(
    point=True
).encode(
    x='date',
    y=altair.Y('value', scale=altair.Scale(zero=False)),
    color='stationName',
    tooltip=['value', 'stationName', altair.Tooltip('date', format='%Y-%m-%d %H:%M')]
).facet(
    column='parameterId',
).resolve_scale(
    y='independent'
).interactive(bind_y=False)

# diplay the chart
line_chart

In [1]:
import hkvportal

In [2]:
hkvportal.__version__

'0.2'

In [ ]:
export_html = 'output.html'
line_chart.save(export_html)

In [ ]:
with open(export_html, 'r') as file:
    data = file.read()

In [4]:
from hkvportal.io.services import dataportal as dp
dp = dp()

In [11]:
dp.setDataservice(dataservice = 'https://dmws.hkvservices.nl/dataportal/')
#dp.setDataservice(dataservice = 'http://amoer/dataservices/')

dataservice is set. https://dmws.hkvservices.nl/dataportal/ will be used as portal


In [ ]:
dp.createDatabase(database = 'WaterDataScheldestromen')

In [12]:
# dp.setEntryDatabase(database = 'WaterDataScheldestromen', 
#                     key = 'stuwen_csv_data', 
#                     data = timeseries.to_csv(),
#                     description = 'waterstand tijdreeks van 3 stuwen')

gg = dp.getEntryDatabase(database = 'WaterDataScheldestromen', 
                    key = 'stuwen_csv_data', content_type='application/csv')

https://dmws.hkvservices.nl/dataportal/data.asmx/read2?database=WaterDataScheldestromen&key=stuwen_csv_data&contentType=application/csv


In [14]:
gg

,Value cannot be null.
0,Parameter name: value


In [15]:
# dp.setEntryDatabase(database = 'WaterDataScheldestromen', 
#                     key = 'stuwen_series', 
#                     data = data, 
#                     description = 'waterstand tijdreeks van 3 stuwen')

dp.getEntryDatabase(database = 'WaterDataScheldestromen', 
                    key = 'stuwen_series', content_type='text/html')

https://dmws.hkvservices.nl/dataportal/data.asmx/read2?database=WaterDataScheldestromen&key=stuwen_series&contentType=text/html


b'<!DOCTYPE html><html><head>  <style>    .vega-actions a {        margin-right: 12px;        color: #757575;        font-weight: normal;        font-size: 13px;    }    .error {        color: red;    }  </style>  <script type="text/javascript" src="https://cdn.jsdelivr.net/npm//vega@5"></script>  <script type="text/javascript" src="https://cdn.jsdelivr.net/npm//vega-lite@3.2.1"></script>  <script type="text/javascript" src="https://cdn.jsdelivr.net/npm//vega-embed@4"></script></head><body>  <div id="vis"></div>  <script>      var spec = {"config": {"view": {"width": 400, "height": 300}, "mark": {"tooltip": null}}, "data": {"name": "data-f662ee9b6eec36757e49acb14e495670"}, "facet": {"column": {"type": "nominal", "field": "parameterId"}}, "spec": {"mark": {"type": "line", "point": true}, "encoding": {"color": {"type": "nominal", "field": "stationName"}, "tooltip": [{"type": "quantitative", "field": "value"}, {"type": "nominal", "field": "stationName"}, {"type": "temporal", "field": "dat

In [ ]:
url = 'http://amoer/dataservices/data.asmx/read?database=WaterDataScheldestromen&key=stuwen_series'
r = requests.get(url)

In [ ]:
r.headers['Content-Type'].split(';')[0]

In [ ]:
r.headers

In [ ]:
import requests